In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter
import re

df = pd.read_csv('berkeley_2019_sampled.csv')
df


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Unnamed: 0,body,emo_pred_pos,emo_pred_neu,emo_pred_neg
0,0,They tried to cut my catalytic converter off w...,0.3639,0.4793,0.1568
1,1,With extremely expensive minerals inside.,0.0153,0.2908,0.6939
2,2,"Not gonna lie, I had to google what a Catalyti...",0.1438,0.6108,0.2454
3,3,I never have figured out why some cross-listed...,0.1038,0.5491,0.3472
4,4,Yes it’s possible but most EE/CS TA’s are not ...,0.0513,0.5154,0.4333
...,...,...,...,...,...
28394,28394,You cannot use the swap feature for discussion...,0.0227,0.3030,0.6743
28395,28395,phase 1 ended so you need to wait for your pha...,0.0420,0.6904,0.2676
28396,28396,They try to look at your performance relative ...,0.1984,0.6447,0.1569
28397,28397,Reserve a room in mainstacks?,0.0444,0.9044,0.0512


In [17]:
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    filtered_words = [word for word in text.split() if word not in stop_words]
    return ' '.join(filtered_words)

df['filtered_body'] = df['body'].astype(str).apply(remove_stopwords)
df[['filtered_body']].head()

,filtered_body
0,tried cut catalytic converter parked brazilian...
1,extremely expensive minerals inside
2,gonna lie google catalytic converter wikipedia...
3,never figured crosslisted classes different st...
4,yes possible eecs tas full time hour tas whose...


In [18]:
df = df.drop(columns=['emo_pred_pos', 'emo_pred_neu', 'emo_pred_neg'])
df

,Unnamed: 0,body,filtered_body
0,0,They tried to cut my catalytic converter off w...,tried cut catalytic converter parked brazilian...
1,1,With extremely expensive minerals inside.,extremely expensive minerals inside
2,2,"Not gonna lie, I had to google what a Catalyti...",gonna lie google catalytic converter wikipedia...
3,3,I never have figured out why some cross-listed...,never figured crosslisted classes different st...
4,4,Yes it’s possible but most EE/CS TA’s are not ...,yes possible eecs tas full time hour tas whose...
...,...,...,...
28394,28394,You cannot use the swap feature for discussion...,cannot use swap feature discussion times chang...
28395,28395,phase 1 ended so you need to wait for your pha...,phase ended need wait phase open
28396,28396,They try to look at your performance relative ...,try look performance relative perceived rigor ...
28397,28397,Reserve a room in mainstacks?,reserve room mainstacks


In [42]:
nrc = pd.read_csv('NRC-Emotion-Lexicon-Wordlevel-v0.92.txt',
                  sep='\t',
                  names=['word', 'emotion', 'association'],
                  encoding=  'iso-8859-1')

#nrc = nrc[nrc['association'] == 1]

from collections import defaultdict
emotion_dict = defaultdict(list)
for _, row in nrc.iterrows():
    emotion_dict[row['word']].append(row['emotion'])

nrc

,word,emotion,association
0,NaN,NaN,q/0083;62e1aa1b;Safari;181DA220-8668-43D6-AC3A...


In [35]:
emotions = sorted(nrc['emotion'].unique())

def analyze_emotions(text):
    words_list = text.split()
    emotion_counts = dict.fromkeys(emotions, 0)
    for word in words_list:
        if word in emotion_dict:
            for emotion in emotion_dict[word]:   
                emotion_counts[emotion] += 1
    return pd.Series(emotion_counts)

emotion_scores = df['filtered_body'].apply(analyze_emotions)
df = pd.concat([df, emotion_scores], axis=1)
df 



,Unnamed: 0,body,filtered_body
0,0,They tried to cut my catalytic converter off w...,tried cut catalytic converter parked brazilian...
1,1,With extremely expensive minerals inside.,extremely expensive minerals inside
2,2,"Not gonna lie, I had to google what a Catalyti...",gonna lie google catalytic converter wikipedia...
3,3,I never have figured out why some cross-listed...,never figured crosslisted classes different st...
4,4,Yes it’s possible but most EE/CS TA’s are not ...,yes possible eecs tas full time hour tas whose...
...,...,...,...
28394,28394,You cannot use the swap feature for discussion...,cannot use swap feature discussion times chang...
28395,28395,phase 1 ended so you need to wait for your pha...,phase ended need wait phase open
28396,28396,They try to look at your performance relative ...,try look performance relative perceived rigor ...
28397,28397,Reserve a room in mainstacks?,reserve room mainstacks
